# Desarrollo
## Avances
- Cargadas las señales en formato MNE y preprocesadas, falta revisar si funciona correctamente

## Conclusiones
- Hay que ver como cargar el archivo de montage para MNE.

In [1]:
#LIBRERÍAS

#Librerías de Sistema
import os
import sys
import time

#Librerías de Manejo de Datos
import pandas as pd
import numpy as np
import scipy.signal as sgn
import scipy.stats as sts
from numbers import Number #usando isinstance(X, Number) puedo detectar si X es una variable numerica de cualquier tipo

#Librerías de Interfaz
from tkinter import filedialog
from tkinter import *
from tkinter import messagebox

#Librerías para EEG
import mne

#Librerías para Plotings
import matplotlib.pyplot as plt

import plotly.graph_objs as go
import plotly.offline as py
import plotly

py.init_notebook_mode(connected = True) #Estableciendo modo Offline para Plotly

### Variables Globales 

In [2]:
#definimos las variables de antemano para que las use todo el algoritmo, como globales

sample_rate = 0.0 #frecuencia de muestreo fundamental de la señal (cuantos datos por segundo se tomaron)

fq_lo = 0      #frequencia baja del filtro pasabanda
fq_hi = 0      #frecuencia alta del filtro pasabanda
fq_no = 0      #frecuencia del filtro notch
fq_sub = 0     #frecuencia de submuestreo

int_quant = 0  #Cantidad de intervalos a definir
int_name= []   #Lista de nombres de los intervalos
int_ini= []    #Lista de inicios en segundos de cada intervalo
int_end= []    #Lista de fines en segundos de cada intervalo
int_ini_sample= [] #Lista de inicios en muestras de cada intervalo
int_end_sample= [] #Lista de fines en muestras de cada intervalo


In [3]:
#Definir Layouts para Plotly

#Layout para plot en tiempo
time_layout = go.Layout(title = "Will be changed in each plot",
                        xaxis = dict(title = 'Time [s]',
                                     rangeslider = dict(visible = True,
                                                        yaxis = dict(rangemode = 'auto'
                                                                    )
                                                        )
                                    ),
                        yaxis = dict(title = 'Amplitude [uV]',
                                     fixedrange = False)
                       )

#Layout para plot en frecuencia
freq_layout = go.Layout(title = "Will be changed in each plot",
                        xaxis = dict(title = 'Frequecy [Hz]',
                                     range = [0, 60],
                                     rangeslider = dict(visible = True,
                                                        yaxis = dict(rangemode = 'auto'
                                                                    )
                                                        )
                                    ),
                        yaxis = dict(title = 'Amplitude [uV]',
                                     fixedrange = False)
                       )

#Layout para plot de espectrograma
spec_layout = go.Layout(title = "Will be changed in each plot",
                        xaxis = dict(title = 'Time [s]'
                                    ),
                        yaxis = dict(title = 'Frequency [Hz]',
                                     range = [0, 60],
                                     fixedrange = False)
                       )

### Funciones

In [4]:
#Función utilizada para obtener los índices de elementos de una lista

get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y]

#This would work like : X = ["Mario", "Luigi", "Yoshi", "Mario"]
#print(get_indexes("Mario", X)) -> Out: [0,3]
#print(get_indexes("Luigi", X)) -> Out: [1]

In [5]:
#Función utilizada para generar una ventana que haga una pregunta "Sí/No"

def yn_quest (quest):
    yn_quest_win = Tk()
    
    answer = messagebox.askquestion("QUESTION:",quest)
    yn_quest_win.destroy()
    
    yn_quest_win.mainloop()
    
    return answer


In [6]:
#Función usada para cargar la señal

def load_signal():   
    nom_s = input("Wich will be the signal name? ") #Pedimos el nombre de la señal

    while True:
        search_signal_win = Tk()                                             #Crea el ambiente de interfaz TK
        xs = filedialog.askopenfilename(title = "Select Signal File :",          #Dice que va a sacar la variable xs como el filename de un archivo
                                        filetypes = (("Documento de texto","*.txt"),
                                                     ("Excel (post 2003)", ".xlsx"),
                                                     ("Excel (prev 2003)", ".xls"),
                                                     ("Todos los archivos","*.*")
                                                    )                        #Con titulo de ventana y opciones de abrir archivos de ese tipo
                                       )

        search_signal_win.destroy()   #Destuimos el ambiente de Tk luego de utilizada la ventana
        search_signal_win.mainloop()  #Establecemos que la ventana quede abierta hasta que se de una respuesta

        print (xs)

        #Detección de tipo de archivo, en este momento el programa sólo puede leer archivos txt y excel
        if ".txt" in xs:
            opt = yn_quest('Is it a file exported by the OpenBCI Software?') #Si es txt, puede haber sido exportado por OpenBCI o no
            if opt == 'Yes' or opt == 'yes':
                XS, col_names, col_types, usecols, skiprows, sep = openbci_topd(xs)
            else:
                XS, col_names, col_types, usecols, skiprows, sep = csv_topd(xs)
            print("Signal Loaded. TXT format.")
            break
        
        elif ".xlsx" in xs or ".xls" in xs:
            XS, col_names, col_types, usecols, skiprows, sep = excel_topd(xs, interval = False)
            print("Signal Loaded. Excel format.") #Esto sería reemplazado por la función "leer txt" que tomaría la dirección y parámetros como Cantidad de columnas, Nombres de columnas, etc (o se puede probar con la direccion sola y si no funciona pedir parametros)
            break

        #Si la persona presiona cancelar, en xs se carga un "", por lo que usamos esa condición     
        elif xs == "":
            print("Cancelled Operation")
            break  #Al presionar cancelar salimos del loop que abre la ventana de seleccion de archivo

        #Si se selecciona un archivo que no sea txt o excel, se aclara que el formato no era correcto, pero no sale del loop
        else:
            print("Wrong File Format. Try Again.")
    
    return XS, nom_s, col_names, col_types, usecols, skiprows, sep

In [7]:
#Función que transforma un txt (CSV) en DataFrame (tabla) de Pandas

def csv_topd(truepath):
    
    #Creamos un loop que pruebe cargar el archivo CSV con una cierta configuración, y sino solicite los datos nuevamente
    while True:
        col_names, col_types, usecols, skiprows, sep = col_rows_sep()         #Función que solicita al usuario ingresar los datos necesarios para cargar el archivo CSV
        
        try:
            X = pd.read_csv(truepath,  sep=sep, skiprows=skiprows, names=col_names, usecols=range(usecols), lineterminator = '\n')
        
        #Testeamos que los valores ingresados en las columnas de tipo "Channel", "Accel" y "Tag" (que son las que vamos a usar) sean numericas, en realidad solo esta testeando que el primer valor de dicahs columnas sea numerica, tenemos que ver si podemos mejorarlo
            for j in range(len(col_types)):
                if col_types[j] == "Channel" or col_types[j] == "Tag" or col_types[j] == "Accel":
                    assert isinstance(X.values[0,j], Number) and pd.notnull(X.values[0,j]) == True
 
        
        except AssertionError:
            print("\n\nThere's a non-numerical value in a column corresponding to Channel, Accel, or Tag, wich should be numerical")
        
        except:
            print('\n\nThe input parameters do not match. Please enter them again.\n\n')
            
        else:
            save_opt = input('Do you wish to save this configuration?: ') #Todavía no lo implementamos, está de recordatorio, sería poder guardar la configuración de parámetros de carga de un archivo para poder usarla luego
            break
    
    X.col_names = col_names #Esto sería medio de gusto si ahora col_names y col_types son globales, pero quedó.
    X.col_types = col_types
    
    return X, col_names, col_types, usecols, skiprows, sep

In [8]:
#Función que transforma un txt (CSV) exportado por OpenBCI en DataFrame (tabla) de Pandas

def openbci_topd(truepath):
        
    selecc_predet = yn_quest('Will you use the default OpenBCI configuration?') #Esta opción setea los parámetros de carga a los que usa OpenBCI predeterminadamente, que se ven abajo
    
    if selecc_predet == 'No' or selecc_predet == 'no': #si decide que no, el usuario ingresa los parámetros a utilizar para la carga
        col_names, col_types, usecols, skiprows, sep = col_rows_sep()
        
    elif selecc_predet == 'Yes' or selecc_predet == 'yes': #si decide que si, los parámetros utilizados son los siguientes
        
        sep = ","  #Coma como separador de columnas
        col_names = ['Sample', 'Ch1', 'Ch2', 'Ch3', 'Ch4', 'x', 'y', 'z', 'Time', ' '] #Nombre de las columnas
        col_types = ['Misc', 'Channel', 'Channel', 'Channel', 'Channel', 'Accel', 'Accel', 'Accel', 'Time', 'Misc'] #Tipo de valores que contiene cada columna
        usecols = 10
        skiprows = 6

    while True:        
                
        try:
            X = pd.read_csv(truepath,  sep=sep, skiprows=skiprows, names=col_names, usecols=range(usecols), lineterminator = '\n')
        
        #Testeamos que los valores ingresados en las columnas de tipo "Channel", "Accel" y "Tag" (que son las que vamos a usar) sean numericas, en realidad solo esta testeando que el primer valor de dicahs columnas sea numerica, tenemos que ver si podemos mejorarlo
            for j in range(len(col_types)):
                if col_types[j] == "Channel" or col_types[j] == "Tag" or col_types[j] == "Accel":
                    assert isinstance(X.values[0,j], Number) and pd.notnull(X.values[0,j]) == True
        
        except AssertionError:
            
            print("\n\nThere's a non-numerical value in a column corresponding to Channel, Accel, or Tag, wich should be numerical")
            
            col_names, col_types, usecols, skiprows, sep = col_rows_sep()         #Función que solicita al usuario ingresar los datos necesarios para cargar el archivo CSV
                
        except:
            print('\n\nThe input parameters do not match. Please enter them again.\n\n')
            
            col_names, col_types, usecols, skiprows, sep = col_rows_sep()         #Función que solicita al usuario ingresar los datos necesarios para cargar el archivo CSV
            
        else:
            if selecc_predet == 'No' or selecc_predet == 'no':
                save_opt = input('Do you wish to save this configuration?: ') #Todavía no lo implementamos, está de recordatorio, sería poder guardar la configuración de parámetros de carga de un archivo para poder usarla luego
            break 
    
    return X, col_names, col_types, usecols, skiprows, sep

In [9]:
#Función que transforma un Excel en DataFrame (tabla) de Pandas, dicho excel puede contener la señal o los datos de los intervalos

def excel_topd(truepath, interval = True): #La segunda variable, interval, define si el arvhivo excel a leer es de intervalos o de señal
    
    while True:
        #Si es un excel de intervalos, se pregunta si dicho excel tiene una columna de Endpoint o de duracion de los intervalos, y sino se solicita que el usuario ingrese un valor de duración general para todos los intervalos
        if interval == True:
            
            header = 0
            index_col = 0
            usecols = 3
            skiprows = 0
            col_names = ['Name', 'Startpoint']

            endpoint_opt = yn_quest("Does it has an endpoint column?")
            if endpoint_opt == 'Yes' or endpoint_opt == 'yes':
                col_names.append('Endpoint')
                usecols =+ 1

            else:
                duration_opt = yn_quest("Does it has a duration column?")
                if duration_opt == 'Yes' or duration_opt == 'yes':
                    col_names.append('Duration')
                    usecols =+ 1
        
        #Si no es un excel de intervalos se usa la función col_rows_sep para definir los parametros de carga
        elif interval == False:
            header = None
            index_col = None
            col_names, col_types, usecols, skiprows, sep = col_rows_sep()
        
        try:
            X = pd.read_excel(truepath, usecols = range(usecols), skiprows = skiprows, names = col_names, header=header, index_col=index_col)
            
            #Testeamos que los valores ingresados en las columnas de tipo "Channel", "Accel" y "Tag" (que son las que vamos a usar) sean numericas, en realidad solo esta testeando que el primer valor de dicahs columnas sea numerica, tenemos que ver si podemos mejorarlo
            for j in range(len(col_types)):
                if col_types[j] == "Channel" or col_types[j] == "Tag" or col_types[j] == "Accel":
                    assert isinstance(X.values[0,j], Number) and pd.notnull(X.values[0,j]) == True
        
        except AssertionError:
            print("\n\nThere's a non-numerical value in a column corresponding to Channel, Accel, or Tag, wich should be numerical")
        
        except:
            print('\n\nThe input parameters do not match. Please enter them again.\n\n')
        
        else:
            break
    
    return X, col_names, col_types, usecols, skiprows

In [10]:
def col_rows_sep():
    
    def accept_col(columnas, col_n, col_t, window):
        window.destroy()
                
        col_names = []
        col_types = []
        
        for i in range(columnas.get()): 
            col_names.append(col_n[i].get()) #El comando get permite obtener de una variable Tk a una variable numpy que se pueda leer numericamente
            col_types.append(col_t[i].get())
            
        return col_names, col_types
                            
    def selec_pd (columnas): #Función llevada a cabo al presionar aceptar en la ventana selec_pd_win       
        
        #Primero levantamos una ventana para confirmar los valores seleccionados
        conf = messagebox.askquestion("Confirm","Will you select : " + str(columnas.get()) + " columns & " + str(filas.get()) + " rows?")
        if conf == "yes":
                        
            #Si se confirman los valores, se destruye la ventana selec_pd_win, y se abre una ventana para dar nombres y tipos a cada columna
            selec_pd_win.destroy()
            
            col_names_win = Tk()
            
            col_params = []
            col_n = []
            col_t = []
            col_names_win.title("Columns")
            col_names_win.label1 = Label(col_names_win, text = "Name:").place(x=30,y=20)
            col_names_win.label2 = Label(col_names_win, text = "Type:").place(x=170,y=20)
            
            col_names_win.geometry("350x"+str(40+columnas.get()*35))
            for i in range(columnas.get()):
                col_n.append(StringVar())
                col_t.append(StringVar())
                col_names_win.entry = Entry(col_names_win, textvariable=col_n[i]).place(x=30 , y=40+i*26)
                col_names_win.spinbox = Spinbox(col_names_win, values = ("Channel", "Misc", "Tag", "Time", "Accel"), wrap =True, textvariable =col_t[i]).place(x=170 , y=40+i*26)
            col_names_win.button1 = Button (col_names_win , text="Accept", command=lambda: col_params.append(accept_col(columnas, col_n, col_t, col_names_win))).place(x=100, y=40+26*columnas.get()) #command= imprimir)
            col_names_win.mainloop()
            
            return col_params[0]
        
    selec_pd_win = Tk() #En este punto comienza la función, creando la ventana de interfaz
    
    col_params = []
    
    #Creamos variables tk para los valores de cantidad de columnas, filas y separador
    columnas = IntVar()
    filas = IntVar()
    separador = StringVar()
    
    #Damos las características de la ventana, titulo, tamaño y componentes(botones, espacios de input, etc)
    selec_pd_win.title("Set parameters for Pandas" ) 
    selec_pd_win.geometry("350x200")

    selec_pd_win.label1 = Label(selec_pd_win , text= "Rows to Skip:").place(x=30,y=20)
    selec_pd_win.entry1 = Entry(selec_pd_win, textvariable=filas).place(x=170 , y=20)

    selec_pd_win.label2 = Label(selec_pd_win, text="Columns to use:").place(x=30 , y=60)
    selec_pd_win.entry2 = Entry(selec_pd_win, textvariable=columnas).place(x=170 , y=60)

    selec_pd_win.label3 = Label(selec_pd_win, text="Separator to use:").place(x=30 , y=100)
    selec_pd_win.entry3 = Entry(selec_pd_win, textvariable=separador).place(x=170 , y=100)
    
    #Al presionar el boton de aceptar, se activa la función selec_pd
    selec_pd_win.button1 = Button (selec_pd_win , text="Accept", command=lambda: col_params.append(selec_pd(columnas))).place(x=170 , y=150) #command= imprimir)

    selec_pd_win.mainloop()
    
    return col_params[0][0], col_params[0][1], columnas.get(), filas.get(), separador.get()

In [11]:
#Función para seleccionar que canalaes se quieren utilizar, y qué posición tiene cada uno

def chan_config(col_names, col_types):
    
    def selec_chan(chan_s, chan_p):
        chan_pos = []
        chan_selecc = []
        
        chan_selec_win.destroy()
        for i in range(chan_quant):
            chan_pos.append(chan_p[i].get())
            chan_selecc.append(chan_s[i].get())
            
        return chan_pos, chan_selecc
    
    chan_quant = 0
    chan_all = []
    chan_p = []
    chan_s = []
    
    chan_params = []
    
    i = 0
    for word in col_types:
        if word == 'Channel':
            chan_all.append(col_names[i])
            chan_quant += 1
        i += 1

    chan_selec_win = Tk()
    chan_selec_win.title("Channel Selection")
    chan_selec_win.label1 = Label(chan_selec_win, text = str(chan_quant) +' Channels were detected, wich ones will you use?').place(x=30,y=10)
    
    chan_selec_win.geometry("500x"+str(80+chan_quant*35))
    for i in range(chan_quant):
        chan_s.append(IntVar())
        chan_p.append(StringVar())

        chan_selec_win.label2 = Label(chan_selec_win, text = chan_all[i] + " column:").place(x=20 , y=40+i*26)
        chan_selec_win.check1 = Checkbutton(chan_selec_win, text = "Import?", variable=chan_s[i], onvalue=1, offvalue=0).place(x=90 , y=40+i*26)
        chan_selec_win.label3 = Label(chan_selec_win, text = "Channel Position:").place(x=180 , y=40+i*26)
        chan_selec_win.entry1 = Entry(chan_selec_win, textvariable = chan_p[i]).place(x=300 , y=40+i*26)

    chan_selec_win.button1 = Button (chan_selec_win , text="Accept Channels", command= lambda: chan_params.append(selec_chan(chan_s, chan_p))).place(x=100, y=60+26*chan_quant) #command= imprimir)

    chan_selec_win.mainloop()
    
    return chan_params[0][0], chan_params[0][1], chan_quant

In [12]:
#Función que transforma la variable Pandas de la cual se leyó la señal previamente, en un arreglo Numpy

def pd_toarray(X_pd):
    
    i = 0
    j = 0
    chan_list = []
    accel_list = []
    tag_list = []
    
    #Leyendo las columnas del archivo pandas
    while i < X_pd.shape[-1]:
        
        #guardaremos aquellas del tipo Channel en una variable
        if col_types[i] == "Channel":
            if chan_selecc[j] == 1:
                chan_list.append(X_pd.columns[i])
            j += 1
        
        #las del tipo Accel en otra, por si en algún momento sirve utilizar los datos del acelerómetro
        if col_types[i] == " Accel":
            accel_list.append(X_pd.columns[i])
        
        #y en otra variable las del tipo Tag, para cuando se necesiten para definir intervalos
        if col_types[i] == "Tag":
            tag_list.append(X_pd.columns[i])
        i += 1

    X_chan = (X_pd.filter(chan_list)).values
    X_accel = (X_pd.filter(accel_list)).values
    X_tag = (X_pd.filter(tag_list)).values
    
    return X_chan, X_accel, X_tag

In [13]:
#Función que define la frecuencia de muestreo fundamental (sample_rate)

def find_samplerate(X_pd):
    
    time_col_index = get_indexes("Time", col_types)
    
    if len(time_col_index) == 1: #Algorithm to try to detect the sample rate from the Time type column
        
        try:
            
            assert(":" in X_pd.values[0, time_col_index[0]])
            
        except:
            
            freq_detect = 1 / (float(X_pd.values[1, time_col_index[0]]) - float(X_pd.values[0, time_col_index[0]])) 
        
        else:
            
            (hh0, mm0, ss0) = X_pd.values[0, time_col_index[0]].split(":")
            (hh1, mm1, ss1) = X_pd.values[1, time_col_index[0]].split(":")
            t0 = 3600 * float(hh0) + 60 * float(mm0) + float(ss0)
            t1 = 3600 * float(hh1) + 60 * float(mm1) + float(ss1)
            
            if (t1-t0) != 0:
                freq_detect = 1 / (t1-t0)
            
            else:
                time_col_index = []
            
    samplerate_win = Tk()
    samplerate_opt = IntVar()
    samplerate_entry = DoubleVar()
    
    sample_rate_params = []

    def freqw_destroy (samplerate_opt, samplerate_entry, freq_detect):
        
        if samplerate_entry.get() != 0 and samplerate_entry.get() != 0.0:
            samplerate_opt.set(1)

        if samplerate_opt.get() == 0 :
            samplerate_conf = messagebox.askquestion("Confirm","Will you select " + str(freq_detect) + " Hz (detected)?")
            sample_rate = freq_detect
            
        elif samplerate_opt.get() == 1 :
            samplerate_conf = messagebox.askquestion("Confirm","Will you select " + str(samplerate_entry.get()) + " Hz (entered)?")
            sample_rate = samplerate_entry.get()
            
        if samplerate_conf == "yes":
            samplerate_win.destroy()
            
        return sample_rate

    samplerate_win.title("Set sample rate" ) 
    samplerate_win.geometry("350x150")

    if len(time_col_index) == 1:
        
        Radiobutton(samplerate_win , variable=samplerate_opt , value=0).place(x=300,y=30)
        samplerate_win.label1 = Label(samplerate_win , text = ' The detected sample rate is ' + str (freq_detect) + ' Hz').place(x=10,y=30)

    if len(time_col_index) == 0: #If there's more than one column with the Time type, then it conflicts with automated detection of samplerate - should be changed later to conflict when there's no Time columns, but when there are more than one, should be choosable
        
        samplerate_win.label1 = Label(samplerate_win , text = ' No Time type columns, unable to detect SampleRate.').place(x=10,y=30)
        samplerate_opt.set(1)
    
    if len(time_col_index) > 1:
        
        samplerate_win.label1 = Label(samplerate_win , text = ' Too many columns with Time type, no SampleRate detection.').place(x=10,y=30)
        samplerate_opt.set(1)
    
    Radiobutton(samplerate_win , variable=samplerate_opt , value=1).place(x=300,y=70)
    samplerate_win.label2 = Label(samplerate_win , text = ' Enter manually : ').place(x=10,y=70)
    samplerate_win.entry1 = Entry(samplerate_win , textvariable= samplerate_entry ).place(x=130 , y=70)

    samplerate_win.button1 = Button(samplerate_win , text="Accept", command=lambda: sample_rate_params.append(freqw_destroy(samplerate_opt, samplerate_entry, freq_detect))).place(x=150, y=110)

    samplerate_win.mainloop()
    
    return sample_rate_params[0]

In [14]:
#Función que transforma arreglo Numpy en formato MNE, que nos permitirá manipular datos EEG, EMG, etc

def array_tomneRaw(X_chan):
    global chan_selecc
    global chan_pos
    ch_names = []
    #montage = []

    for i in get_indexes(1,chan_selecc):
        ch_names.append('Ch'+ str(i+1))
        #montage.append(chan_pos[i])   No funciona así, porque hay que darle una variable montage especifica, hay que ver como crearla
    
    #Debemos crear una variable con la información de la señal, nombre y tipo de los canales, y posición
    X_mneinfo = mne.create_info(ch_names = ch_names, 
                                ch_types = 'eeg', 
                                #montage = montage,  ver como crear el montage https://martinos.org/mne/dev/generated/mne.channels.Montage.html#mne.channels.Montage
                                sfreq = sample_rate) 
    
    #Creamos la variable MNE, transponiendo la matriz de datos numpy ya que MNE posiciona los canales como filas
    #Se puede acceder a la información dentro de la variable MNE a traves de sus atributos X_mneRaw.get_data(), X_mneRaw.ch_names(), etc
    X_mneRaw = mne.io.RawArray(np.transpose(X_chan), info = X_mneinfo)
    
    return X_mneRaw 


In [15]:
def tag_interval(tag_colindex):

    global int_ini
    global int_end
    global int_ini_sample
    global int_end_sample
    global int_name
    global fq_sub
    
    int_ini_sample = []
    int_end_sample = []
    int_name = []

    a = X_pd.values[:, tag_colindex]

    tags = np.unique(a)

    tag_names = []

    for t in tags:

        print("Wich is the name for the tag '" + str(t) + "' \n")
        tag_name = input("Entry:")
        tag_names.append(tag_name)

        first = True
        n = 1
        for i in range(a.shape[0]):

            if a[i] ==  t and first == True:

                int_ini_sample.append(i)
                int_name.append(tag_names[get_indexes(t, tags)[0]] + " " + str(n))
                n = n + 1

                first = False

            elif a[i] != t and first == False:

                int_end_sample.append(i-1)
                first = True

            elif a[i] == t and i == (a.shape[0]-1):
                int_end_sample.append(i)

    for i in int_ini_sample:
        int_ini.append(int(i/fq_sub))
        
    for e in int_end_sample:
        int_end.append(int(e/fq_sub))

In [16]:
#Función que permite definir los intervalos a utilizar, con nombres, inicios y finales

def interval_choice ():  
    
    global int_quant
    global int_name
    global int_ini
    global int_end
    global int_ini_sample
    global int_end_sample
    global fq_sub
    
    int_quant = 0
    int_name= []
    int_ini= []
    int_end= []
    int_ini_sample= []
    int_end_sample= []

    if len(get_indexes("Tag", col_types)) == 1:
        
        tag_int_opt = yn_quest("Type Tag Column Detected, do you wish to use it to define Intervals?")
        
        if tag_int_opt == "Yes" or tag_int_opt == "yes":
            tag_interval(tag_colindex = get_indexes("Tag", col_types)[0])
            excel_int_opt = "no"
    
        else:
            excel_int_opt= yn_quest("Do you wish to load an excel file with intervals?")
    
    #Si el usuario desea, se puede cargar un archivo de excel con los intervalos ya definidos
    if excel_int_opt == "Yes" or excel_int_opt == "yes":  
        
        search_int_win = Tk() #Crea la ventana root
        int_path = filedialog.askopenfilename(title = "Select Signal File :",          #Dice que va a sacar la variable Xs como el filename de un archivo
                                        filetypes = (("Excel (post 2003)", ".xlsx"),
                                                     ("Excel (prev 2003)", ".xls"),
                                                     ("All files","*.*")
                                                    ) #Con titulo de ventana y opciones de abrir archivos de ese tipo
                                       ) 
        search_int_win.destroy()
        
        int_pd = excel_topd(int_path)
        
        int_quant = int_pd.shape[0]
        
        for i in range(int_quant):
        
            int_name.append(int_pd.values[i,0])
            int_ini.append(int_pd.values[i,1])
            
            if 'Endpoint' in int_pd.columns:
                int_end.append(int_pd.values[i,2])
            
            elif 'Duration' in int_pd.columns:
                int_end.append(int_pd.values[i,1] + int_pd.values[i,2])
            
            else:
                if i == 0:
                    int_durat = int(input("How long did the intervals last (in sec)?: "))
                int_end.append(int_pd.values[i,1] + int_durat)
                
            sample_ini= (int_ini[i]*int(fq_sub))
            sample_end= (int_end[i]*int(fq_sub))

            int_ini_sample.append(sample_ini)
            int_end_sample.append(sample_end)
        
        search_int_win.mainloop()
    
    #Si no, se pueden definir manualmente
    if excel_int_opt == "No" or excel_int_opt =="no" and tag_int_opt == "No" or tag_int_opt == "no":
        
        time_layout.title = "Entire Run"
            
        int_fig = go.Figure(data = [go.Scatter(x = X_mneProc.times, #check
                                               y = X_mneProc.get_data()[0, :]
                                              )],
                            layout = time_layout)
            
        py.iplot(int_fig)
        
        time.sleep(2) #We set it to wait for the plot to be done, and the we ask how many intervals to analyze
        
        int_quant = int(input("How many interval will you analyze?: "))
        i = 0

        #Stablish names and sizes
        while i < int_quant:
            print("Name for the interval no. ", (i+1))
            int_name.append(input())
            #Solicito al usuario los rangos de tiempo para cada intervalo
            print("Start time for interval " + int_name [i] , end ="\n")
            int_ini.append(int(input('Enter in secs: '))) 
            print("End time for interval " + int_name [i] , end ="\n")
            int_end.append(int(input('Enter in secs: '))) 
            #Multiplico esos tiempos por la frecuencia de submuestreo, para obtener la cantidad de muestras que quiero ver
            sample_ini= (int_ini[i]*int(fq_sub))
            sample_end= (int_end[i]*int(fq_sub))

            int_ini_sample.append(sample_ini)
            int_end_sample.append(sample_end)
            i = i + 1
            
    print(int_name)
    print(int_ini)
    print(int_end)
    print(int_ini_sample)
    print(int_end_sample)
    print(fq_sub)
    
#A futuro: implementar una funcipon que permita guardar un excel con los intervalos cargados manualmente, y otra función que defina los intervalos a partir de los tags

In [17]:
#Función para el preprocesado de la señal X_mneRaw

def sgn_preproc(X_mneRaw):
    global fq_lo
    global fq_hi
    global fq_no
    global fq_sub
    
    #Solicitamos orden de filtro, el predeterminado es 5 ya que es el que suele usarse normalmente en EEG
    filt_order = input("Insert Butterworth filter Order (if blank, default is 5): ")
    
    if filt_order == '':
        filt_order = 5
    else:
        filt_order = int(filt_order)
     
    #Solicitamos la frecuencia inferior de filtro
    fq_lo = input("Insert Low Frequency for Bandpass Filter (leave blank space for no filtering): ")
    
    #Si no se ingresa nada, se supone que no se usará frecuncia de corte inferior (filtro pasabajo)
    if fq_lo == '':
        fq_lo = None
    else:
        fq_lo = float(fq_lo)
    
    #Solicitamos la frecuencia superior de filtro
    fq_hi = input("Insert High Frequency for Bandpass Filter (leave blank space for no filtering): ")
    
    #Si no se ingresa nada, se supone que no se usará frecuncia de corte superior (filtro pasaalto)
    if fq_hi == '':
        fq_hi = None
    else:
        fq_hi = float(fq_hi)
    
    #Creamos una copia de la señal, sino al tratar de filtrarla, filtraremos tambien la señal Raw
    filtered_signal = X_mneRaw.copy()
    
    #Este if significaría que si al menos una de las dos (fq_hi o fq_lo) tiene valor, entonces aplicamos filtro
    if fq_lo != None or fq_hi != None:
        filtered_signal.filter(l_freq = fq_lo,
                               h_freq = fq_hi,
                               method = 'iir',
                               iir_params = dict(order = filt_order,
                                                 ftype = "butter"
                                                )
                            )
    
    #Si ambas son None, no filtramos
    else:
        print('No Bandpass Filter applied')
    
    #Solicitamos la frecuencia de filtro de Notch
    fq_no = input("Insert Frequency for Notch Filter (leave blank space for no filtering): ") #Se puede agregar para que filtre en varias frecuencias con notch, pasando una lista de floats
    
    #Si no se ingresa nada, se supone que no se usará filtro Notch
    if fq_no == '':
        fq_no = None
    else:
        fq_no = float(fq_no)
    
    if fq_no != None:
        filtered_signal.notch_filter(freqs = fq_no,
                                    notch_widths = 1,
                                    method = 'iir',
                                    iir_params = dict(order = filt_order,
                                                      ftype = "butter"
                                                     )
                                    )
    
    else:
        print('No Notch Filter applied')
    
    #Solicitamos la frecuencia de submuestreo
    
    message = str(sample_rate) + ' Hz is the main sample rate, would you like to Subsample?'
    
    while True:
        fq_sub_opt = yn_quest(message)
        
        if fq_sub_opt == "yes" or fq_sub_opt == "Yes":
            
            fq_sub = input("Insert Subsample Frequency (Must be lower than "  + str(sample_rate) + "): ")
            
            if float(fq_sub) < sample_rate:
                fq_sub = float(fq_sub)
                filtered_signal.resample(sfreq = fq_sub)
                break
            
            else:
                message = 'Entered Subsample Frequency is greater than the Manin Sample Rate, would you like to subsample?'
        
        if fq_sub_opt == "no" or fq_sub_opt == "No":
            
            fq_sub = float(sample_rate)
            break
    
    return filtered_signal
                
    
#    #Si no se ingresa nada, se supone que no se usará Submuestreo
#    if fq_sub != '' and fq_sub != str(sample_rate) and fq_sub != str(int(sample_rate)):
#        fq_sub = float(fq_sub)
#        filtered_signal.resample(sfreq = fq_sub)
#        return filtered_signal
#    else:
#        fq_sub = sample_rate
#        return filtered_signal

In [18]:
#Función que grafica las señales en tiempo, actualmente de todos los intervalos, luego se implementaría la selección de algunos intervalos

def plot_time ():
    
    plot_raw_opt = yn_quest('Do you wish to plot the Raw Signal?')
    plot_proc_opt = yn_quest('Do you wish to plot the Processed Signal?')
    
    for i in range(int_quant):  #Iria en vez de int_quant algún int_selecc que indique solo intervalos seleccionados
        raw_scat = []
        proc_scat = []
 
        for j in range(X_mneRaw.get_data().shape[0]):
            raw_scat.append(go.Scatter(x = np.arange(int_ini[i], int_end[i], 1/fq_sub),
                                       y = X_mneRaw.get_data()[j, int_ini_sample[i]:int_end_sample[i]],
                                       name = X_mneRaw.ch_names[j] #cambiar despues por posicion en montage, cuando montage este implementado
                                      )
                           )
            proc_scat.append(go.Scatter(x = np.arange(int_ini[i], int_end[i], 1/fq_sub),
                                        y = X_mneProc.get_data()[j, int_ini_sample[i]:int_end_sample[i]],
                                        name = X_mneProc.ch_names[j] #cambiar despues por posicion en montage, cuando montage este implementado
                                       )
                           )
            
            
        
        if plot_raw_opt == 'Yes' or plot_raw_opt == 'yes': 
            time_layout.title = int_name[i] + ' Raw'
            
            raw_fig = go.Figure(data = raw_scat, layout = time_layout)
            
            py.iplot(raw_fig)
            
        if plot_proc_opt == 'Yes' or plot_proc_opt == 'yes':
            time_layout.title = int_name[i] + ' Processed'
            
            proc_fig = go.Figure(data = proc_scat, layout = time_layout)
    
            py.iplot(proc_fig)
        

In [19]:
#Función que grafica las señales en frecuencia, actualmente de todos los intervalos, luego se implementaría la selección de algunos intervalos

def plot_freq (): #Ver si hace falta mostrar la FFT de Raw data signal
    
    plot_fft_opt = yn_quest("Do you wish to plot de FFT signal?")
    
    for i in range(int_quant):  #Iria en vez de int_quant algún int_selecc que indique solo intervalos seleccionados
        fft_scat = []
 
        for j in range(X_mneRaw.get_data().shape[0]):
            fft_scat.append(go.Scatter(x =FFT_matrixProc[i][0],
                                       y = FFT_matrixProc[i][j+1],
                                       name = X_mneRaw.ch_names[j] #cambiar despues por posicion en montage, cuando montage este implementado
                                      )
                           )
         
            
        freq_layout.title = int_name[i]
            
        if plot_fft_opt == 'Yes' or plot_fft_opt == 'yes':
            
            fft_fig = go.Figure(data = fft_scat, layout = freq_layout)
            py.iplot(fft_fig)


In [20]:
#Función que grafica espectrogramas, actualmente de todos los intervalos, luego se implementaría la selección de algunos intervalos

def plot_spec ():
    
    plot_spec_opt = yn_quest("Do you wish to plot de spectrogram of the signal?")
    
    for i in range(int_quant):  #Iria en vez de int_quant algún int_selecc que indique solo intervalos seleccionados
        
        for j in range(X_mneRaw.get_data().shape[0]):
            spec_heatmap = go.Heatmap(x = Spec_matrixProc[i][0],
                                      y = Spec_matrixProc[i][1],
                                      z = Spec_matrixProc[i][j+2],
                                      colorscale='Viridis',
                                     )         
            
            spec_layout.title = int_name[i] + ' ' + X_mneRaw.ch_names[j]
            
            if plot_spec_opt == 'Yes' or plot_spec_opt == 'yes':
            
                spec_fig = go.Figure(data = [spec_heatmap], layout = spec_layout)
            
                py.iplot(spec_fig)

In [21]:
#Función que genera las fft de la señal en cada intervalo y cada canal, colocandolas a todas en una unica variable para poder acceder a ellas la momento de graficar

def sgn_fft(signal):
    
    fft_list = [] #Esta lista contendra un arreglo de varias listas, una por cada intervalo, cada una con las transformadas de los canales seleccionados para cada intervalo
    
    #Se solicita al usuario si quiere utilizar Welch (transformada con ventana en trasladada) o transformada FFT normal
    welch_opt = yn_quest("Will you use Welch's method for tranforming?")
    
    if welch_opt == 'Yes' or welch_opt == 'yes':
        #Si se acepta el uso de Welch, se solicitan sus parámetros
        welch_twin = int(input('How long shall the transform window be (in secs)?: '))
        welch_twin_overlap = int(input('How many seconds will be overlapped?: '))
        
        for i in range(int_quant):
            
            each_fft_list = [] #En cada iteración se cargará esta lista con los residuos de frecuencia y las transformadas de cada canal, para un cierto intervalo
            
            for j in range(signal.shape[0]):
                
                fr, tf = sgn.welch(signal[j, int_ini_sample[i]:int_end_sample[i]], 
                                   fs=fq_sub, 
                                   nperseg = int(fq_sub)*welch_twin, 
                                   noverlap = int(fq_sub)*welch_twin_overlap,
                                   scaling = "spectrum"
                                  )
                if j == 0: #En la primera iteración se guardan los residuos de frecuencia, iguales para todos los canales, y luego cada una de las transformadas
                    each_fft_list.append(fr)
                
                each_fft_list.append(tf)
                
            #De esta manera queda una lista formada each_fft_list[0]=residuos de frecuencia para el intervalo i, each_fft_list[<0]=transformada para cada canal seleccionadopara el intervalo i
            #Esta lista se agrega a la lista general y luego se resetea para el proximo intervalo.
            fft_list.append(each_fft_list)
    
    else:
        
        for i in range(int_quant):
            
            each_fft_list = [] 
            
            fr = np.fft.fftfreq(int_end_sample[i]-int_ini_sample[i], d = 1/fq_sub)
            each_fft_list.append(fr) #en el caso de FFT, funciona similar solo que no se solicitan parámetros de Welch, y se agregan los resiudos por separado, ya que np.fft tiene un comando para ello.
            
            for j in range(signal.shape[0]):
                
                tf = np.fft.fft(signal[j, int_ini_sample[i]:int_end_sample[i]])
                each_fft_list.append(abs(tf)/(int_end_sample[i]-int_ini_sample[i]))
            
            fft_list.append(each_fft_list)
                                
    
    return(fft_list)       

In [22]:
#Función que genera las fft de la señal en cada intervalo y cada canal, colocandolas a todas en una unica variable para poder acceder a ellas la momento de graficar

def sgn_spec (signal) :
    
    spec_list = [] #Similar al caso de las FFT, esta lista contendra un arreglo con listas para cada intervalo, donde cada lista posee los residuos de tiempo y frecuencia, y los espectrogramas para cada canal
        
    spec_twin = int(input('How long shall the spectrogram window be (in secs)?: '))
    spec_twin_overlap = int(input('How many seconds will be overlapped?: '))
        
    for i in range(int_quant):
            
        each_spec_list = [] #En cada iteración se cargaran en esta lista los resiudos de tiempo y frecuencia, y los espectrogramas de cada canal, para un cierto intervalo i
            
        for j in range(signal.shape[0]):
            Freq_spec , Time_spec , Values_spec = sgn.spectrogram (x=signal[j,int_ini_sample[i]:int_end_sample[i]],
                                                                   fs=fq_sub ,
                                                                   nperseg= spec_twin*int(fq_sub),
                                                                   noverlap= spec_twin_overlap*int(fq_sub),
                                                                   scaling = "spectrum"
                                                                  )
                                                                
            if j == 0:
                each_spec_list.append(Time_spec)
                each_spec_list.append(Freq_spec)
                
            each_spec_list.append(Values_spec)
                
        spec_list.append(each_spec_list)
    
    return spec_list

In [23]:
#Función de calculo de la correlación entre los canales de la señal, para cada intervalo

def sgn_correlation ():
    index = []

    chan_num = X_mneRaw.get_data().shape[0] 
    
    #Creamos dos matrices que reciban los datos de correlacion (rho y pvalor) de tamaño (cant de canales)^2 x (cant de intervalos) ya que se cruzan cada canal con todos los otros en cada intervalo 
    rho_list = np.zeros((chan_num**2, int_quant))  
    pval_list = np.zeros((chan_num**2, int_quant))

    n = 0
    for i in range(chan_num): #Recorremos la combinación de cada canal i con cada canal j
        for j in range(chan_num):

            for k in range(int_quant): #Recorremos todos los intervalos con k
                rho, pval = sts.spearmanr(X_mneProc.get_data()[i , int_ini_sample[k]:int_end_sample[k]], 
                                          X_mneProc.get_data()[j , int_ini_sample[k]:int_end_sample[k]],
                                          axis = 0
                                         )
                rho_list[n, k] = rho
                pval_list[n, k] = pval


            n += 1 #ya que las filas estan indicadas por i y j a la vez, lo más simple es crear un puntero n que apunte a cada posicion
            
            #en cada vuelta agregamos al index los canales que se estan correlacionando
            index.append(str(X_mneRaw.ch_names[i]) + ' vs ' + str(X_mneRaw.ch_names[j])) #cambiaremos ch_names por montage
 
    #Creamos las variables pandas Dataframe con las matrices de datos, index como nombres de filas (canal vs canal) y nombres de intervalos para cada columna
    cor_rho_pd = pd.DataFrame(data = rho_list, columns=int_name , index = index)
    cor_pval_pd = pd.DataFrame(data = pval_list, columns=int_name , index = index)
    
    display(cor_rho_pd)
    display(cor_pval_pd)

    return cor_rho_pd , cor_pval_pd 

In [24]:
#Función de calculo de la coherencia entre los canales de la señal, para cada intervalo

def sgn_coherence ():
    
    index= []

    chan_num = X_mneRaw.get_data().shape[0] 
    
    #Creamos una matriz que reciba los datos de correlacion media, de tamaño (cant de canales)^2 x (cant de intervalos) ya que se cruzan cada canal con todos los otros en cada intervalo 
    coh_mean_list = np.zeros((chan_num**2, int_quant))

    n = 0
    
    for i in range(chan_num):
        for j in range(chan_num):

            for k in range(int_quant):

                f, cxy = sgn.coherence(x = X_mneProc.get_data()[i , int_ini_sample[k]:int_end_sample[k]],
                                       y = X_mneProc.get_data()[j , int_ini_sample[k]:int_end_sample[k]],
                                       fs = fq_sub,
                                       nperseg = 2*fq_sub,
                                       noverlap = 1*fq_sub,
                                       axis = 0
                                   )

                coh_mean_list[n, k] = np.mean(cxy) #En cada posicion cargamos la media de coherencia

            n += 1 

            index.append(str(X_mneRaw.ch_names[i]) + ' vs ' + str(X_mneRaw.ch_names[j])) #cambiaremos ch_names por montage

    coh_mean_pd = pd.DataFrame(data = coh_mean_list, columns=int_name, index = index)

    display(coh_mean_pd)
    
    return coh_mean_pd

### Cuerpo del Código

In [25]:
X_pd, signalname, col_names, col_types, usecols, skiprows, sep = load_signal()

chan_pos, chan_selecc, chan_quant = chan_config(col_names, col_types)

X_chan, X_accel, X_tag = pd_toarray(X_pd)

sample_rate = find_samplerate(X_pd)

#X_mneRaw = array_tomneRaw(X_chan)

#X_mneProc = sgn_preproc(X_mneRaw)

#interval_choice() 

#plot_time()

#FFT_matrixRaw = sgn_fft(X_mneRaw.get_data())

#FFT_matrixProc = sgn_fft(X_mneProc.get_data())

#plot_freq()

#Spec_matrixProc = sgn_spec(X_mneProc.get_data())

#plot_spec()

#Rho_pd , Pval_pd = sgn_correlation ()

#Coh_pd = sgn_coherence()

Wich will be the signal name? agus
C:/Users/Usuario/Desktop/96cb-Fase1-AR1-18-01-19.txt
Signal Loaded. TXT format.
